# Combinación de conjuntos de datos: Concat y Append

Algunos de los estudios de datos más interesantes provienen de la combinación de diferentes fuentes de datos.
Estas operaciones pueden implicar cualquier cosa, desde la concatenación muy directa de dos conjuntos de datos diferentes, hasta uniones y fusiones más complicadas al estilo de las bases de datos que manejan correctamente cualquier solapamiento entre los conjuntos de datos.
Las ``Series`` y los ``DataFrame`` están construidos con este tipo de operaciones en mente, y Pandas incluye funciones y métodos que hacen que este tipo de manipulación de datos sea rápida y sencilla.

Aquí veremos una simple concatenación de ``Series`` y ``DataFrame`` con la función ``pd.concat``; más tarde nos sumergiremos en fusiones y uniones más sofisticadas en memoria implementadas en Pandas.

Comenzamos con las importaciones estándar:

Traducción realizada con la versión gratuita del traductor www.DeepL.com/Translator

In [ ]:
import pandas as pd
import numpy as np

Por comodidad, definiremos esta función que crea un ``DataFrame`` de una forma particular que será útil más adelante:

In [ ]:
def make_df(cols, ind):
    """Quickly make a DataFrame"""
    data = {c: [str(c) + str(i) for i in ind]
            for c in cols}
    return pd.DataFrame(data, ind)

# example DataFrame
make_df('ABC', range(3))

,A,B,C
0,A0,B0,C0
1,A1,B1,C1
2,A2,B2,C2


Además, crearemos una clase rápida que nos permita mostrar múltiples ``DataFrame`` uno al lado del otro. El código hace uso del método especial ``_repr_html_``, que IPython utiliza para implementar su visualización de objetos enriquecidos:

In [ ]:
class display(object):
    """Display HTML representation of multiple objects"""
    template = """<div style="float: left; padding: 10px;">
    <p style='font-family:"Courier New", Courier, monospace'>{0}</p>{1}
    </div>"""
    def __init__(self, *args):
        self.args = args
        
    def _repr_html_(self):
        return '\n'.join(self.template.format(a, eval(a)._repr_html_())
                         for a in self.args)
    
    def __repr__(self):
        return '\n\n'.join(a + '\n' + repr(eval(a))
                           for a in self.args)
    

El uso de esto se aclarará a medida que continuemos nuestra discusión en la siguiente sección.

## Recall: Concatenación de arrays NumPy

La concatenación de objetos ``Series`` y ``DataFrame`` es muy similar a la concatenación de arrays de Numpy, que se puede hacer a través de la función ``np.concatenate`` como se discute en [The Basics of NumPy Arrays](02.02-The-Basics-Of-NumPy-Arrays.ipynb).
Recuerda que con ella puedes combinar el contenido de dos o más arrays en un único array:

In [ ]:
x = [1, 2, 3]
y = [4, 5, 6]
z = [7, 8, 9]
np.concatenate([x, y, z])

array([1, 2, 3, 4, 5, 6, 7, 8, 9])

El primer argumento es una lista o tupla de arrays a concatenar.
Además, toma una palabra clave ``axis`` que permite especificar el eje a lo largo del cual se concatenará el resultado:

In [ ]:
x = [[1, 2],
     [3, 4]]
np.concatenate([x, x], axis=1)

array([[1, 2, 1, 2],
       [3, 4, 3, 4]])

## Concatenación simple con ``pd.concat``

Pandas tiene una función, ``pd.concat()``, que tiene una sintaxis similar a ``np.concatenate`` pero contiene una serie de opciones que discutiremos momentáneamente:

```python
# Signature in Pandas v0.18
# Firma en Pandas v0.18
pd.concat(objs, axis=0, join='outer', join_axes=None, ignore_index=False,
          keys=None, levels=None, names=None, verify_integrity=False,
          copy=True)
```

``pd.concat()`` puede utilizarse para una simple concatenación de objetos ``Series`` o ``DataFrame``, al igual que ``np.concatenate()`` puede utilizarse para simples concatenaciones de arrays:

In [ ]:
ser1 = pd.Series(['A', 'B', 'C'], index=[1, 2, 3])
ser2 = pd.Series(['D', 'E', 'F'], index=[4, 5, 6])
pd.concat([ser1, ser2])

1    A
2    B
3    C
4    D
5    E
6    F
dtype: object

También funciona para concatenar objetos de mayor dimensión, como ``DataFrame``:

In [ ]:
df1 = make_df('AB', [1, 2])
df2 = make_df('AB', [3, 4])
display('df1', 'df2', 'pd.concat([df1, df2])')

,A,B
1,A1,B1
2,A2,B2
,A,B
3,A3,B3
4,A4,B4
,A,B
1,A1,B1
2,A2,B2
3,A3,B3
4,A4,B4


Por defecto, la concatenación se realiza por filas dentro del ``DataFrame`` (es decir, ``axis=0``).
Al igual que ``np.concatenate``, ``pd.concat`` permite especificar un eje a lo largo del cual se producirá la concatenación.
Considere el siguiente ejemplo:

In [ ]:
df3 = make_df('AB', [0, 1])
df4 = make_df('CD', [0, 1])
display('df3', 'df4', "pd.concat([df3, df4], axis='col')")

df3
    A   B
0  A0  B0
1  A1  B1

df4
    C   D
0  C0  D0
1  C1  D1

pd.concat([df3, df4], axis='col')
    A   B   C   D
0  A0  B0  C0  D0
1  A1  B1  C1  D1

Podríamos haber especificado de forma equivalente ``axis=1``; aquí hemos utilizado el más intuitivo ``axis='col'``. 

### Índices duplicados

Una diferencia importante entre ``np.concatenate`` y ``pd.concat`` es que la concatenación en Pandas *preserva los índices*, ¡aunque el resultado tenga índices duplicados!
Considere este sencillo ejemplo:

In [ ]:
x = make_df('AB', [0, 1])
y = make_df('AB', [2, 3])
y.index = x.index  # make duplicate indices!
display('x', 'y', 'pd.concat([x, y])')

,A,B
0,A0,B0
1,A1,B1
,A,B
0,A2,B2
1,A3,B3
,A,B
0,A0,B0
1,A1,B1
0,A2,B2
1,A3,B3


Observe los índices repetidos en el resultado.
Aunque esto es válido dentro de ``DataFrame``, el resultado es a menudo indeseable.
``pd.concat()`` nos da unas cuantas maneras de manejarlo.

#### Atrapar las repeticiones como un error

Si quieres simplemente verificar que los índices en el resultado de ``pd.concat()`` no se solapan, puedes especificar la bandera ``verify_integrity``.
Si se establece como True, la concatenación lanzará una excepción si hay índices duplicados.
A continuación se muestra un ejemplo, en el que, para mayor claridad, capturaremos e imprimiremos el mensaje de error:

In [ ]:
try:
    pd.concat([x, y], verify_integrity=True)
except ValueError as e:
    print("ValueError:", e)

ValueError: Indexes have overlapping values: [0, 1]


#### Ignorar el índice

A veces el índice en sí mismo no importa, y se prefiere que simplemente se ignore.
Esta opción se puede especificar utilizando la bandera ``ignorar_índice``.
Si se establece como verdadero, la concatenación creará un nuevo índice entero para la ``Serie`` resultante:

In [ ]:
display('x', 'y', 'pd.concat([x, y], ignore_index=True)')

,A,B
0,A0,B0
1,A1,B1
,A,B
0,A2,B2
1,A3,B3
,A,B
0,A0,B0
1,A1,B1
2,A2,B2
3,A3,B3


#### Añadir claves MultiIndex

Otra opción es utilizar la opción ``keys`` para especificar una etiqueta para las fuentes de datos; el resultado será una serie indexada jerárquicamente que contiene los datos:

In [ ]:
display('x', 'y', "pd.concat([x, y], keys=['x', 'y'])")

x
    A   B
0  A0  B0
1  A1  B1

y
    A   B
0  A2  B2
1  A3  B3

pd.concat([x, y], keys=['x', 'y'])
      A   B
x 0  A0  B0
  1  A1  B1
y 0  A2  B2
  1  A3  B3

El resultado es un ``DataFrame`` multi-indexado, y podemos utilizar las herramientas discutidas en [Indexación jerárquica](03.05-Indexación-jerárquica.ipynb) para transformar estos datos en la representación que nos interesa.

### Concatenación con joins

En los ejemplos sencillos que acabamos de ver, hemos concatenado principalmente ``DataFrame`` con nombres de columna compartidos.
En la práctica, los datos de diferentes fuentes pueden tener diferentes conjuntos de nombres de columnas, y ``pd.concat`` ofrece varias opciones en este caso.
Considera la concatenación de los siguientes dos ``DataFrame``, que tienen algunas (pero no todas) columnas en común:

In [ ]:
df5 = make_df('ABC', [1, 2])
df6 = make_df('BCD', [3, 4])
display('df5', 'df6', 'pd.concat([df5, df6])')

df5
    A   B   C
1  A1  B1  C1
2  A2  B2  C2

df6
    B   C   D
3  B3  C3  D3
4  B4  C4  D4

pd.concat([df5, df6])
     A   B   C    D
1   A1  B1  C1  NaN
2   A2  B2  C2  NaN
3  NaN  B3  C3   D3
4  NaN  B4  C4   D4

By default, the entries for which no data is available are filled with NA values.
To change this, we can specify one of several options for the ``join`` and ``join_axes`` parameters of the concatenate function.
By default, the join is a union of the input columns (``join='outer'``), but we can change this to an intersection of the columns using ``join='inner'``:

In [ ]:
display('df5', 'df6',
        "pd.concat([df5, df6], join='inner')")

df5
    A   B   C
1  A1  B1  C1
2  A2  B2  C2

df6
    B   C   D
3  B3  C3  D3
4  B4  C4  D4

pd.concat([df5, df6], join='inner')
    B   C
1  B1  C1
2  B2  C2
3  B3  C3
4  B4  C4

Otra opción es especificar directamente el índice de las columnas restantes utilizando el argumento ``join_axes``, que toma una lista de objetos índice.
Aquí especificaremos que las columnas devueltas deben ser las mismas que las de la primera entrada:

In [ ]:
display('df5', 'df6',
        "pd.concat([df5, df6], join_axes=[df5.columns])")

,A,B,C
1,A1,B1,C1
2,A2,B2,C2
,B,C,D
3,B3,C3,D3
4,B4,C4,D4
,A,B,C
1,A1,B1,C1
2,A2,B2,C2
3,NaN,B3,C3
4,NaN,B4,C4


La combinación de opciones de la función ``pd.concat`` permite una amplia gama de comportamientos posibles al unir dos conjuntos de datos; téngalos en cuenta cuando utilice estas herramientas para sus propios datos.

### El método ``append()``

Debido a que la concatenación directa de arrays es tan común, los objetos ``Series`` y ``DataFrame`` tienen un método ``append`` que puede lograr lo mismo con menos pulsaciones.
Por ejemplo, en lugar de llamar a ``pd.concat([df1, df2])``, puedes simplemente llamar a ``df1.append(df2)``:

In [ ]:
display('df1', 'df2', 'df1.append(df2)')

,A,B
1,A1,B1
2,A2,B2
,A,B
3,A3,B3
4,A4,B4
,A,B
1,A1,B1
2,A2,B2
3,A3,B3
4,A4,B4


Tenga en cuenta que a diferencia de los métodos ``append()`` y ``extend()`` de las listas de Python, el método ``append()`` de Pandas no modifica el objeto original, sino que crea un nuevo objeto con los datos combinados.
Tampoco es un método muy eficiente, porque implica la creación de un nuevo índice *y* un buffer de datos.
Por lo tanto, si planeas hacer múltiples operaciones de ``append``, generalmente es mejor construir una lista de ``DataFrame`` y pasarlos todos a la vez a la función ``concat()``.

En la siguiente sección, veremos otro enfoque más potente para combinar datos de múltiples fuentes, las fusiones/uniones de estilo base de datos implementadas en ``pd.merge``.
Para más información sobre ``concat()``, ``append()``, y otras funcionalidades relacionadas, vea la sección ["Merge, Join, and Concatenate"](http://pandas.pydata.org/pandas-docs/stable/merging.html) de la documentación de Pandas.